# In this lesson, we will

- Study the `JsonHttpAction` Sample Manifest File
- Use the `requests` library to create a Pet against our local server (api-server-one)
- Create a JsonHttpAction Manifest File to create a Pet
- Run the framework and examine the Report

# Step 1: Examine the Template json_http_action.yaml

In [3]:
!cat /tmp/api_client_one/manifests/actions/json_http_action.yaml

model_name: JsonHttpActionModel
config:
  method:
    format: string
    template: GET
  params:
    format: yaml
    template: |
      abc: abc
  url:
    format: string
    template: "http://www.abc.com"
description: Dummy Json Http Request
execution_id: 'json_http_action'
schemas:
  - id: "json_200"
    model_name: JsonSchemaModel
    schema_:
      format: json
      text: |
        {
          "$schema": "http://json-schema.org/draft-07/schema#",
          "type": "object",
          "additionalProperties": false
          }

schema_validators:
  - id: "json_200"
    model_name: JsonSchemaValidatorModel
    schema_id: "200"
    against: 'output_body'


# Explanation of JsonHttpAction Manifest

- *model_name*: A mandatory field which indicates which underlying model we are using. `JsonHttpActionModel` refers to a Model which communicates using JSON as body over HTTP. 

- *config*: This section contains all the user-input parameters relevant to the model. All config items are jinja-templateable. Examples are method (Http Method), params, URL, body and headers. All configurable items require `format` (as in what it should be converted to) and `template`.

- *description*: A description of what the Action is about

- *execution_id*: This ID must be unique in a ScenarioModel. A ScenarioModel contains a series of ActionModels. This will be discussed in another lesson. When left blank, the execution_id of an action manifest will take the same name as the name of the manifest file.

- *schemas*: A list of either JsonSchema or XmlSchema specific to this ActionModel. Can be an empty list.

- *schema_validators*: A list of mapping which maps any one schema in schemas (specified via the `schema_id` field) with the any field in the output field (auto-generated by the programme, specified via the `against` field)



# Step 2: Programmatically create a Pet entry using requests library

In [7]:
# Create a Pet 
import requests
from datetime import datetime

pet_id = 3
url = f'http://localhost:8080/pets/{pet_id}'
body = {
  "animal_type": "cat",
  "name": "Susie",
  "tags": {}
}

resp = requests.put(url=url,json=body)

print(f"{resp.status_code=}")
print(f"{resp.text=}")
print(f"{resp.headers=}")

resp.status_code=200
resp.text=''
resp.headers={'Server': 'Werkzeug/2.2.3 Python/3.11.5', 'Date': 'Sun, 24 Sep 2023 07:09:34 GMT', 'Content-Type': 'application/json', 'Content-Length': '0', 'Connection': 'close'}


In [ ]:
%load /tmp/api_client_one/manifests/actions/json_http_action.yaml